# Learning FLPO

In this code, we will implement the hierarchical ML architecture to predict the parameters in the FLPO setup.

In [1]:
import torch
from VRP_Net import VRPNet
from matplotlib import pyplot as plt
from utils import *
import os
import matplotlib.pyplot as plt
from torchinfo import summary

# Loading the Data

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Running on: " , device)
num_data = 10
num_cities = 12
city_dim = 2
data = torch.rand(num_data,num_cities,city_dim).to(device)

Running on:  cuda


# Loading the VRP NET Module

In [4]:
hidden_dim = 128
num_layers = 3
num_heads = 8
input_dim = 2
num_samples = 5 # for inference
model = VRPNet(input_dim,hidden_dim,device,num_heads=num_heads,num_enc_layers=num_layers,num_dec_layers=num_layers,use_PE=False)
model.load_state_dict(torch.load('Saved models/POMO2025_03_12 11_03_550.8357473611831665best_model.pth',weights_only=True))
model.eval()
print('VRP NET loaded.')

VRP NET loaded.


# Generating d_min

In [6]:
all_samples = torch.zeros(num_data, num_cities, num_samples).to(device)
for i in range(num_samples):
    _, actions = model(data, mod='train')
    all_samples[:,:,i:i+1] = actions
opt_sample_indices = []
for data_idx in range(num_data):
    costs_for_data = [route_cost(data[data_idx:data_idx+1], all_samples[data_idx:data_idx+1, :, sample_idx]) for sample_idx in range(num_samples)]
    costs_tensor = torch.tensor(costs_for_data)
    d_min, opt_sample_index = torch.min(costs_tensor, dim=0)  
    opt_sample_indices.append(opt_sample_index.item())  
opt_sample_indices = torch.tensor(opt_sample_indices)
actions = all_samples[torch.arange(num_data),:,opt_sample_indices]
print("d_min is: ",d_min)

d_min is:  tensor(0.3225)


# Loading Free Energy Network

Dhananjay's code for loading his DNN is here. The output of this block should be Y.

# Calculating Free Energy

Feeding the d_min from Salar Block to get the Free energy

# Calculating the gradients

The output of this block shoud be dF/dy

# Running CLF to determin Y

The GD code here

# Closing the Beta Loop for Annealing